In [1]:
import pandas as pd

# = = =  = = =  = = =

from urllib.parse import urlparse, parse_qs, urlencode

import requests

import sys
sys.path.append('../00.Tools')
from crawler_configuration import get_header, get_proxy

from bs4 import BeautifulSoup
from lxml import etree

In [2]:
print('Data Loading...')
list_series_error, df_input = [], pd.read_excel('./file/input.xlsx',
                                                header=0,
                                                dtype=str).fillna('')
print('Loading Done !')
print()

df_input['No'] = df_input['No'].astype(int)

crawler_remain = len(df_input)

print(f'总数量：{crawler_remain}')
print()

df_input

Data Loading...
Loading Done !

总数量：2



,No,Url
0,1,https://www.ebay.com/sch/i.html?_dkr=1&iconV2R...
1,2,https://www.ebay.de/sch/i.html?_dkr=1&iconV2Re...


In [3]:
crawler_status, (crawler_index, crawler_series) = False, next(df_input.iterrows())

crawler_status, crawler_index, crawler_series

(False,
 0,
 No                                                     1
 Url    https://www.ebay.com/sch/i.html?_dkr=1&iconV2R...
 Name: 0, dtype: object)

In [4]:
import time

url_parse = urlparse(crawler_series['Url'])

dict_param = parse_qs(url_parse.query)
dict_param.pop('_pgn', None)
dict_param['orig_cvip'] = 'true'
dict_param['_ipg'] = '240'
if crawler_series['Url'].startswith('https://www.ebay.com/'):
    dict_param['_ul'] = 'US'
    dict_param['_stpos'] = '91710'
elif crawler_series['Url'].startswith('https://www.ebay.de/'):
    dict_param['_ul'] = 'DE'
    dict_param['_stpos'] = '10115'
elif crawler_series['Url'].startswith('https://www.ebay.co.uk/'):
    dict_param['_ul'] = 'GB'
    dict_param['_stpos'] = 'BS81QU'
elif crawler_series['Url'].startswith('https://www.ebay.com.au/'):
    dict_param['_ul'] = 'AU'
    dict_param['_stpos'] = '2019'
elif crawler_series['Url'].startswith('https://www.ebay.ca/'):
    dict_param['_ul'] = 'CA'
    dict_param['_stpos'] = 'M5S2E8'

url_base = f'{url_parse.scheme}://{url_parse.netloc}{url_parse.path}?{urlencode(dict_param, doseq=True)}'

list_item_number = []
for crawler_cycle in range(10):

    page = 1
    while True:

        crawler_retry = 0
        while True:
            crawler_retry += 1
            print(crawler_cycle + 1, page, crawler_retry)
            try:
                url_request = f'{url_base}&_pgn={page}'

                resp = requests.get(url_request,
                                    headers=get_header(),
                                    proxies=get_proxy(),
                                    timeout=(5, 15))

                if resp.status_code == 200:
                    soup = BeautifulSoup(resp.text, 'lxml')
                    html = etree.HTML(str(soup))

                    list_item_number_temp = [href.split('?')[0].split('/')[-1].strip() for href in html.xpath('//a[@class="s-item__link"]/@href')]
                    list_item_number_temp = [item_number_temp for item_number_temp in list_item_number_temp if len(item_number_temp) == 12]
                    if list_item_number_temp:
                        if crawler_cycle == 0 and page == 1:
                            with open('./item_number.html', 'w', encoding='UTF-8') as file:
                                file.write(str(soup))
                        break
            except KeyboardInterrupt:
                break
            except:
                time.sleep(3)
                continue

        # = = =  = = =  = = =

        list_item_number.extend(list_item_number_temp)
        list_item_number = list(dict.fromkeys(list_item_number))

        # = = =  = = =  = = =

        print(f' - {len(list_item_number)}')

        # = = =  = = =  = = =

        page += 1

        # = = =  = = =  = = =

        if not html.xpath('//nav[@class="pagination"]/a[@type="next"]'):
            break

list_item_number

1 1 1
 - 240
1 2 1
1 2 2
1 2 3
 - 450
1 3 1
 - 475
2 1 1
 - 491
2 2 1
 - 519
2 3 1
 - 519
3 1 1
 - 522
3 2 1
 - 522
3 3 1
 - 522
4 1 1
 - 522
4 2 1
 - 522
4 3 1
 - 522
5 1 1
 - 522
5 2 1
 - 522
5 3 1
 - 522


['232315264904',
 '255628697516',
 '335513085831',
 '235461387733',
 '334707355204',
 '335372891937',
 '256264372483',
 '332322450335',
 '335058229363',
 '335270737835',
 '235845462212',
 '335058269621',
 '255531675896',
 '235766373358',
 '334041377271',
 '255586148611',
 '235443868102',
 '335057592559',
 '256102649999',
 '255410646663',
 '235653299500',
 '232479816006',
 '253139036562',
 '252900220689',
 '335058230165',
 '235836763979',
 '235263054774',
 '335473437422',
 '235382407258',
 '254598959347',
 '334956110193',
 '235443864456',
 '332388018897',
 '256572886011',
 '235598791204',
 '232315242152',
 '232435277242',
 '256372876426',
 '233621258535',
 '235793317653',
 '256469386720',
 '235544392154',
 '234053013857',
 '234095593488',
 '334041548817',
 '335403954358',
 '235748987359',
 '235801407523',
 '255018389434',
 '232315321167',
 '235443871387',
 '255527869390',
 '234277033709',
 '235055850780',
 '335333813239',
 '235741753580',
 '334151803763',
 '235647554777',
 '335511425815

In [5]:
list_item_number

['232315264904',
 '255628697516',
 '335513085831',
 '235461387733',
 '334707355204',
 '335372891937',
 '256264372483',
 '332322450335',
 '335058229363',
 '335270737835',
 '235845462212',
 '335058269621',
 '255531675896',
 '235766373358',
 '334041377271',
 '255586148611',
 '235443868102',
 '335057592559',
 '256102649999',
 '255410646663',
 '235653299500',
 '232479816006',
 '253139036562',
 '252900220689',
 '335058230165',
 '235836763979',
 '235263054774',
 '335473437422',
 '235382407258',
 '254598959347',
 '334956110193',
 '235443864456',
 '332388018897',
 '256572886011',
 '235598791204',
 '232315242152',
 '232435277242',
 '256372876426',
 '233621258535',
 '235793317653',
 '256469386720',
 '235544392154',
 '234053013857',
 '234095593488',
 '334041548817',
 '335403954358',
 '235748987359',
 '235801407523',
 '255018389434',
 '232315321167',
 '235443871387',
 '255527869390',
 '234277033709',
 '235055850780',
 '335333813239',
 '235741753580',
 '334151803763',
 '235647554777',
 '335511425815

In [6]:
print('Data Outputting...')
df_correct = pd.DataFrame({'No': range(1, len(list_item_number) + 1),
                           'Item Number': list_item_number})
df_correct.to_excel('./test_item_number.xlsx', index=False)

df_correct

Data Outputting...


,No,Url
0,1,232315264904
1,2,255628697516
2,3,335513085831
3,4,235461387733
4,5,334707355204
...,...,...
517,518,235677067106
518,519,255459637046
519,520,335464272743
520,521,335393586358


In [7]:
print('Output Finished !')
print()

Output Finished !



In [8]:
crawler_series['Request_Url'] = url_request

list_series_error.append(crawler_series)

list_series_error

[No                                                             1
 Url            https://www.ebay.com/sch/i.html?_dkr=1&iconV2R...
 Request_Url    https://www.ebay.com/sch/i.html?_dkr=1&iconV2R...
 Name: 0, dtype: object]

In [9]:
if list_series_error:
    df_error = pd.DataFrame(list_series_error).sort_values(by=['No'],
                                                           ascending=[True],
                                                           ignore_index=True)
    df_error.to_excel('./test_item_number_error.xlsx', index=False)
    print('- Error Occurred -')
    print()

df_error if list_series_error else ''

- Error Occurred -



,No,Url,Request_Url
0,1,https://www.ebay.com/sch/i.html?_dkr=1&iconV2R...,https://www.ebay.com/sch/i.html?_dkr=1&iconV2R...


In [10]:
print('Done ~')

Done ~
